# 实验十 霍尔效应及其应用

> Copyright © 2023 [@Maxwell Jay](https://github.com/MaxwellJay256)
> 
> Copyright © 2024 [@Zjl37](https://github.com/Zjl37)
>
> 本文件修改自 [HITSZ-OpenAuto/PHYS1002](https://github.com/HITSZ-OpenAuto/PHYS1002)，遵循与彼仓库同样的许可协议。


## 参考公式


### 亥姆赫兹线圈公共轴线中点的磁感应强度

$$
B_0 = \frac{8 \mu_0 N I_M}{5 ^ {3/2} R}
$$

### 各向异性磁阻传感器

传感器灵敏度定义式：
$$
s = \frac{V_{out}}{(V_s \cdot B)}
$$


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as cnst
import scipy
import csv

def read_csv(filename):
    """读取 csv 文件"""
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        return list(filter(len, reader)) # 去除空行

plt.rc('font', size=9)
plt.rcParams['text.usetex'] = True # 如果你的电脑上没装有 LaTeX 环境或画图仍遇到问题，注释此行

**载流直螺线管的主要参数**

| 总长 | 线圈内径 | 匝数 |
| :--: | :------: | :--: |
| 0.270m | 0.030m | 3200 |


In [ ]:
# 读取数据
data = read_csv('./data/st1-VH-IM-raw.csv')
header = data[0]
data = np.array(data[1:]).transpose()
# 提取数据
IM = data[0].astype(float) # (A)
V = data[[2,3,4,5]].astype(float) # [V1, V2, V3, V4] (mV)

# 参数
n = 3200 # 匝数
L = 0.270 # (m)
D = 0.030 # (m)

IS = 5.000 # (mA)


### 参考公式：螺线管轴线中央的磁场强度
$$
B = \frac {\mu_0 n I_M} {\sqrt {L^2 + D^2}}
$$


In [ ]:
# 计算 V_H
VH = (V[0]-V[1]+V[2]-V[3])/4 # (mV)

# 计算磁感应强度
B = cnst.mu_0 * n / np.hypot(L, D) * IM # (T)
B = 1000*B # mT

In [ ]:
# 保存处理的数据
data[1] = [f'{x:.2f}' for x in B]
data[6] = [f'{x:.2f}' for x in VH]

with open('./data/st1-VH-IM.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data.transpose())

### 参考公式：霍尔元件的灵敏度

$$
K_H = \frac{V_H}{I_S B}
$$


In [ ]:
# 使用最小二乘法计算灵敏度
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(IM, VH) # slope: (mV/A)

# 作 V_H–I_M 图
plt.figure(figsize=(3,2))

def yhat(x):
    return slope*x + intercept
plt.plot([0, 0.8], [yhat(0), yhat(0.8)], color='dodgerblue')

plt.plot(IM, VH, '+', color='navy')

plt.ylabel(r'$V_H\:/\:{\rm mV}$')
plt.xlabel(r'$I_M\:/\:{\rm A}$')
plt.grid()
plt.savefig('./fig/st1-VH-IM.svg', bbox_inches='tight')
plt.show()

print('斜率 slope =', slope, 'mV/A')

slope = slope/1000 # (V/A)
KH1 = slope * np.hypot(L, D) / (IS * cnst.mu_0 * n) # (V/mA/T)
KH1 = 1000*KH1 # (V/A/T)

print('灵敏度 K_H1 =', KH1, 'V/A/T')
print('相关系数 r =', r_value)


## 2. 测量霍尔电压 $V_H$ 与工作电流 $I_S$ 的关系

In [ ]:
# 读取数据
data = read_csv('./data/st2-VH-IS-raw.csv')
header = data[0]
data = np.array(data[1:]).transpose()
# 提取数据
IS = data[0].astype(float)
V = data[[1,2,3,4]].astype(float) # [V1, V2, V3, V4] (mV)

# 计算 V_H
VH = (V[0]-V[1]+V[2]-V[3])/4 # (mV)
# 保存处理的数据
data[5] = [f'{x:.2f}' for x in VH]

with open('./data/st2-VH-IS.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data.transpose())

# 参数
IM = 0.500 # (A)
B = cnst.mu_0 * n * IM / np.hypot(L, D) # (T)

# 使用最小二乘法计算灵敏度
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(IS, VH) # slope: (mV/mA) 即 (V/A)
KH2 = slope / B # (V/A/T)
print('斜率 slope =', slope, 'mV/mA')
print('灵敏度 K_H2 =', KH2, 'V/A/T')
print('相关系数 r =', r_value)

# 画图
plt.figure(figsize=(3, 2))

def yhat(x):
    return slope*x + intercept
plt.plot([0, 10], [yhat(0), yhat(10)], color='orange')

plt.plot(IS, VH, '+', color='red')
plt.ylabel(r'$V_H\:/\:\rm{mV}$')
plt.xlabel(r'$I_S\:/\:\rm{mA}$')
plt.grid()
plt.savefig('./fig/st2-VH-IS.svg', bbox_inches='tight')
plt.show()


综合两次实验，得到霍尔元件的灵敏度 $K_H$：

In [ ]:
KH = (KH1 + KH2) / 2 # V/A/T
print('灵敏度 KH =', KH, 'V/A/T')

## 3. 测量螺线管中磁感应强度 $B$ 的大小及分布情况

In [ ]:
# 读取数据
data = read_csv('./data/st3-VH-X-raw.csv')
header = data[0]
data = np.array(data[1:]).transpose()

# 参数
IM = 0.500 # (A)
IS = 5.000 # (mA)

# 提取数据
x = data[0].astype(float) # (mm)
V = data[[1,2,3,4]].astype(float) # [V1, V2, V3, V4] (mV)
# 计算 V_H
VH = (V[0]-V[1]+V[2]-V[3])/4 # (mV)
# 计算各点磁感应强度
B = VH / (IS * KH) # (T)
B = B*1000 # (mT)

# 保存处理的数据
data[5] = [f'{x:.2f}' for x in VH]
data[6] = [f'{x:.2f}' for x in B]

with open('./data/st3-VH-X.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data.transpose())


In [ ]:
# 使用最小二乘法拟合曲线

def B_fn(x, B0, L, x0, D):
    """modal function for B"""
    """x, x0, L, D: (mm); B0: (mT)"""
    X = x - x0
    return B0*((L-X)/np.hypot(L-X, D/2) + X/np.hypot(X, D/2))

popt, pcov = scipy.optimize.curve_fit(B_fn, x, B)
B0, L, x0, D = popt
print("拟合曲线的参数：")
print("    B0 =", B0, "mT")
print("     L =", L , "mm")
print("    x0 =", x0, "mm")
print("     D =", D , "mm")
print("对称轴 x =", L/2+x0 , "mm")



In [ ]:

# 画图
plt.figure(figsize=(6, 4))

# 拟合曲线
xspc = np.linspace(-10, 310, 512)
Bexp = B_fn(xspc, *popt)
plt.plot(xspc, Bexp, color='limegreen')
# 原始数据点
plt.plot(x, B, 'x', color='darkgreen')
# 对称轴
plt.vlines(L/2+x0, 0, 2*B0, 'yellow', 'dashed')

plt.ylabel(r'$B\:/\:{\rm mT}$')
plt.xlabel(r'$x\:/\:{\rm mm}$')
plt.grid()
plt.savefig('./fig/st3-B-x.svg', bbox_inches='tight')
plt.show()


**亥姆赫兹线圈的主要参数**
| 有效半径 | 匝数 | 线径 |
| :------: | :--: | :--: |
| 0.100m | 310 | 0.6mm |


## 4. 各向异性磁阻传感器（AMR）特性

这部分实验中，B 的单位是 Gs，即高斯。$1 \rm T = 10^4 Gs$

In [ ]:

plt.figure(figsize=(6, 4))
plt.plot(B, Vout, 'o-', color='brown')
plt.ylabel(r'$V_{out} \: / \: \rm mV$')
plt.xlabel(r'$B \: / \: \rm Gs$')
plt.grid()
plt.savefig('./fig/4-Vout-B.svg', bbox_inches='tight')
plt.show()


## 5. 各向异性磁阻传感器（AMR）输出电压与角度之间的关系

In [ ]:
# 读取数据
data = read_csv('./data/st5-Vout-theta.csv')
header = data[0]
data = np.array(data[1:]).transpose()
# 提取数据
theta = data[0].astype(float) # (°)
Vout = data[1].astype(float) # (mV)

# 画 Vout-theta 关系曲线
plt.figure(figsize=(6, 4))
plt.plot(theta, Vout, 'o-', color='purple')
plt.ylabel(r'$V_{out} \: / \: \rm mV$')
plt.xlabel(r'$\theta \: / \: \rm ^{\circ}$')
plt.grid()
plt.savefig('./fig/st5-Vout-theta.svg', bbox_inches='tight')
plt.show()


## 思考题

- 估算本实验所用霍尔片的载流子浓度。

$K_H = \frac{1}{n e d}$，所以 $n = \frac{1}{K_H e d}$。

In [ ]:
# 参数
d = 0.001 # (m)
n = 1 / (KH * cnst.e * d) # m**-3
print('载流子浓度 n =', n, 'm^-3')